In [1]:
# Import the dependencies.
import pandas as pd
import numpy as np
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

# looks for nearest city with geo coordinates
from citipy import citipy

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

68.09848940779264 146.97770988880535
71.08326040903904 71.69322423554965
64.13033276973559 0.7933615740990945
52.87772065358283 -151.4989227181672
-67.31043174664052 119.45480048946604
-34.154841400560116 156.69348907063903
73.69583025163513 37.023902450200154
59.9045178637453 -110.1693770965489
29.401604991503362 29.79395312204835
44.19577055130182 100.5029440971781
-64.32854325758503 -43.76737313459043
70.10776728436048 83.12350461036505
-24.97506326161637 145.1288521393198
33.14900499214454 -70.49835212710302
-42.889966017545255 19.00972238187535
44.02039677639027 -37.693505466217374
-76.69560003739667 141.81383267290926
34.15112099085027 -171.1501188143515
31.50596878450635 -97.63438905752133
18.60450535314571 -175.61651157261784
56.54738913848871 113.71115488714491
77.79665580202419 6.548512922800882
22.41563530220813 7.472408944893374
87.58117991602921 -140.827096964394
49.25688591798439 -6.295084524965688
58.58374798465354 -135.95090748571744
4.895871218949793 130.0462064452994


In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

755

In [5]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=3ef41da726500c6f691a33dccaeb7ff7


In [6]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [8]:
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_condition = city_weather["weather"]["description"]
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_weather_condition})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | belaya gora
City not found. Skipping...
Processing Record 2 of Set 1 | yar-sale
City not found. Skipping...
Processing Record 3 of Set 1 | brae
City not found. Skipping...
Processing Record 4 of Set 1 | kodiak
City not found. Skipping...
Processing Record 5 of Set 1 | albany
City not found. Skipping...
Processing Record 6 of Set 1 | port macquarie
City not found. Skipping...
Processing Record 7 of Set 1 | tumannyy
City not found. Skipping...
Processing Record 8 of Set 1 | yellowknife
City not found. Skipping...
Processing Record 9 of Set 1 | el faiyum
City not found. Skipping...
Processing Record 10 of Set 1 | hovd
City not found. Skipping...
Processing Record 11 of Set 1 | ushuaia
City not found. Skipping...
Processing Record 12 of Set 1 | karaul
City not found. Skipping...
Processing Record 13 of Set 1 | emerald
City not found. Skipping...
Processing Record 14 of Set 1 | hamilton
City not found. Skipping...
Processing Record 15 of Set 1 | bredasdorp
Cit

NameError: name 'city_data_df' is not defined

In [ ]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")